In [ ]:
## simple genai app using langchain and ollama model

In [118]:
import os
from dotenv import load_dotenv

# Load environment variables first
load_dotenv()

# Set LangSmith configuration variables before importing any LangChain components
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ["SIMPLE_LANGCHAIN_PROJECT"] = os.getenv("SIMPLE_LANGCHAIN_PROJECT")

# Verify LangSmith project is set
print("Current LangSmith project:", os.getenv("SIMPLE_LANGCHAIN_PROJECT"))
        

Current LangSmith project: GenAIAPPwithOllama


In [119]:
from langsmith.run_helpers import traceable
from langchain_ollama import ChatOllama #as openai is paid

llm = ChatOllama(model="gemma:2b")  # or llama2, mistral, etc.
# Wrap your function so LangSmith logs it
@traceable(name="ollama-query-wrapped", project_name=os.getenv("SIMPLE_LANGCHAIN_PROJECT"))
def ask_ollama(q: str):
    return llm.invoke(q).content

# Run a query
print(ask_ollama("What is beautiful soup?"))

Beautiful soup is a traditional American dish consisting of noodles and vegetables. It is a light and flavorful soup that is often served with butter or cream. The noodles in beautiful soup can be made from a variety of ingredients, such as wheat, barley, or potato. The vegetables in beautiful soup can include celery, carrots, onions, and potatoes.


In [120]:
# Verify LangSmith connection
from langsmith import Client

client = Client()
print("\nVerifying LangSmith Project Configuration:")
print(f"Current Project: {os.getenv('LANGCHAIN_PROJECT')}")
print("\nAvailable Projects:")
for p in client.list_projects():
    print(f"- {p.name}")


Verifying LangSmith Project Configuration:
Current Project: GenAIAPPwithOPENAI

Available Projects:
- GenAIAPPwithOllama
- GenAIAPPwithOPENAI
- default


In [121]:
#data injetion -> from website we need to scrape the data 
from langchain_community.document_loaders import WebBaseLoader
url = "https://docs.langchain.com/langsmith/architectural-overview"
loader = WebBaseLoader(url)
data = loader.load()
print(f"Loaded {len(data)} documents from {url}")

Loaded 1 documents from https://docs.langchain.com/langsmith/architectural-overview


In [122]:
data

[Document(metadata={'source': 'https://docs.langchain.com/langsmith/architectural-overview', 'title': 'Architectural overview - Docs by LangChain', 'language': 'en'}, page_content='Architectural overview - Docs by LangChainDocs by LangChain home pagePythonSearch...⌘KLangSmithPlatform for LLM observability and evaluationOverviewSetupInstall on KubernetesInstall on DockerInteract with an installationUpgrade an installationConfigure egress for subscription metricsView trace counts across an organizationLangSmith-managed ClickHouseConfigurationConfigure for scaleEnable TTL & data retentionCreate an Ingress for installations (Kubernetes)Mirror images for your installationUse environment variables for model providersTroubleshootingAuthentication & access controlSet up basic authenticationSet up SSO with OAuth2.0 & OIDCCustomize user managementConfigure custom TLS certificatesUse an existing secret for your installation (Kubernetes)Connect external servicesEnable blob storageConnect to an ext

In [123]:
# load data -->docs -->divide our text into  chunks -->vectors -->use vector embeddings--> store in a vector store db
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(data)
print(f"Splited into {len(documents)} chunks")

Splited into 9 chunks


In [124]:
documents

[Document(metadata={'source': 'https://docs.langchain.com/langsmith/architectural-overview', 'title': 'Architectural overview - Docs by LangChain', 'language': 'en'}, page_content='Architectural overview - Docs by LangChainDocs by LangChain home pagePythonSearch...⌘KLangSmithPlatform for LLM observability and evaluationOverviewSetupInstall on KubernetesInstall on DockerInteract with an installationUpgrade an installationConfigure egress for subscription metricsView trace counts across an organizationLangSmith-managed ClickHouseConfigurationConfigure for scaleEnable TTL & data retentionCreate an Ingress for installations (Kubernetes)Mirror images for your installationUse environment variables for model providersTroubleshootingAuthentication & access controlSet up basic authenticationSet up SSO with OAuth2.0 & OIDCCustomize user managementConfigure custom TLS certificatesUse an existing secret for your installation (Kubernetes)Connect external servicesEnable blob storageConnect to an ext

In [125]:
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
embeddings = OllamaEmbeddings(model="gemma:2b")  # or llama
vectorstoredb = FAISS.from_documents(documents, embeddings)
print(f"Created vectorstore with {vectorstoredb.index.ntotal} vectors")


Created vectorstore with 9 vectors


In [126]:
vectorstoredb

In [127]:
#query from vector store db
query = "Need to expose the LangSmith frontend"
result_docs = vectorstoredb.similarity_search(query, k=6)
print(f"Found {len(result_docs)} similar documents.")

Found 6 similar documents.


In [128]:
result_docs[0].page_content

'LangSmith uses PostgreSQL as the primary data store for transactional workloads and operational data (almost everything besides traces and feedback).\n\u200bRedis\nRedis is a powerful in-memory key-value database that persists on disk. By holding data in memory, Redis offers high performance for operations like caching.\nLangSmith uses Redis to back queuing and caching operations.\n\u200bBlob storage\nLangSmith supports several blob storage providers, including AWS S3, Azure Blob Storage, and Google Cloud Storage.\nLangSmith uses blob storage to store large files, such as trace artifacts, feedback attachments, and other large data objects. Blob storage is optional, but highly recommended for production deployments.\n\u200bServices\n\u200bLangSmith frontend\nThe frontend uses Nginx to serve the LangSmith UI and route API requests to the other servers. This serves as the entrypoint for the application and is the only component that must be exposed to users.\n\u200bLangSmith backend'

In [129]:
#retrieval chain and document chain
# document chain --> how to combine the document and prompt together, it is responsible for giving me context information.
# retrieval chain --> how to retrieve the relevant documents based on the query
# llm --> the language model we are using to generate the answer

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(
    """
    Answer the following question based only on the provided context:
    <context>
      {context}
    </context>

     """       
)
document_chain = create_stuff_documents_chain(llm=llm, prompt=prompt_template)
document_chain


RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following question based only on the provided context:\n    <context>\n      {context}\n    </context>\n\n     '), additional_kwargs={})])
| ChatOllama(model='gemma:2b')
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [130]:
from langchain_core.documents import Document 
document_chain.invoke(
    {
        "input_documents": result_docs, 
        "context": [Document(page_content = "To access the LangSmith UI and send API requests, you will need to expose the LangSmith frontend service. " \
        "Depending on your installation method, this can be a load balancer or a port exposed on the host machine." \
        "You can run LangSmith in Kubernetes (recommended) or Docker in a cloud environment that you control." \
        "The frontend uses Nginx to serve the LangSmith UI and route API requests to the other servers. " \
        "This serves as the entrypoint for the application and is the only component that must be exposed to users." \
        "LangSmith uses PostgreSQL as the primary data store for transactional workloads and operational data "\
        "(almost everything besides traces and feedback)." \
        "Redis is a powerful in-memory key-value database that persists on disk. By holding data in memory, " \
        "Redis offers high performance for operations like caching." \
        "LangSmith uses Redis to back queuing and caching operations." \
        "Blob storage LangSmith supports several blob storage providers, including AWS S3, Azure Blob Storage, and Google Cloud Storage." \
        "LangSmith uses blob storage to store large files, such as trace artifacts, feedback attachments, and other large data objects. " \
        "Blob storage is optional, but highly recommended for production deployments." \
        "Services LangSmith frontend. The frontend uses Nginx to serve the LangSmith UI and route API requests to the other servers. " \
        "This serves as the entrypoint for the application and is the only component that must be exposed to users.LangSmith backend")],
        "question": query
    }
)

"Sure, here's the answer to the question:\n\nThe LangSmith frontend service must be exposed to users. It is the only component that must be exposed to users."

In [131]:
# we want the document to come from the retriever we just set up to dynamically 
# select the most relevant documents and pass those in for a given question
## pass input to retriever --> retriever act as an interface --> get output from vectorstoredb

retriever = vectorstoredb.as_retriever(search_kwargs={"k": 6})
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, document_chain) #document_chain provides context

In [132]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000022D7A6DB460>, search_kwargs={'k': 6}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following question based only on the provided context:\n    <context>\n      {context}\n    </context>\n\n     '), additional_

In [133]:
#get the response from the llm
response = retrieval_chain.invoke(
    {
        "input": query
    }
)
response['answer']

'The context does not provide information about the storage services available for LangSmith, so I cannot answer this question from the provided context.'

In [102]:
response

{'input': 'Will need to expose the LangSmith frontend service?',
 'context': [Document(id='08dd7d56-6ad5-49ba-90c7-7c2fb618b817', metadata={'source': 'https://docs.langchain.com/langsmith/architectural-overview', 'title': 'Architectural overview - Docs by LangChain', 'language': 'en'}, page_content='\u200bLangSmith backend\nThe backend is the main entrypoint for CRUD API requests and handles the majority of the business logic for the application. This includes handling requests from the frontend and SDK, preparing traces for ingestion, and supporting the hub API.\n\u200bLangSmith queue\nThe queue handles incoming traces and feedback to ensure that they are ingested and persisted into the traces and feedback datastore asynchronously, handling checks for data integrity and ensuring successful insert into the datastore, handling retries in situations such as database errors or the temporary inability to connect to the database.\n\u200bLangSmith platform backend\nThe platform backend is an

In [ ]:
#from langchain_community.chat_models import Ollama
#from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
#from langchain_core.schema import AIMessage, HumanMessage, SystemMessage